In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import mpl_toolkits.mplot3d.axes3d as p3
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

from scipy import io
import math
import csv
import time


# Data Analysis of Forces on Non-Spherical Intruders

__To Complete:__
- 1.1 Read Data from Binary File 
    - Figure out actual format of file
    - numParicles vs. time
    - Store 2D array using np.fromfile 


- 1.2 Plot Data
    - 1.2.1 3D animated scatter plot
    - Fix colormap
    
- Share notebook w Lu
- Read ryan 2.3 and 2.3.4
- divide into 1 row bins, concentration graphs, ignore width, last time frame

## Read Data from Binary File

A file containing 20 variables for N number of particles for time t is to be opened, read, and stored. 

In [ ]:
def readBinDataSE(filename, startN, endN, maxParticles):

    NF=20 #Number of fields in binary file data

    # Data Input
    with open(filename, encoding='latin-1') as file:
        #Encoding is latin-1 !

        for i in range(startN):
            numParticles = np.fromfile(file, float, -1)
            print(numParticles)
            time = np.fromfile(file, float, 1) #2nd float 
            #What is the difference between numParticles and time?
            temp_NF = np.fromfile(file, float, NF)
            temp_numP = np.fromfile(file, float, numParticles)
            temp = []
            temp = temp.append([temp_NF, temp_numP])

        if startN == 1:
            numParticles = len(np.fromfile(file, float, i))
            time = np.fromfile(file, float, -1)
#             f.seek(0)
            
        pData = np.zeros((NF,maxParticles))

        for ii in range(endN-startN):
            numParticles = len(np.fromfile(file, float, -1))
            time = np.fromfile(file, float, -1)
            temp_NF = np.fromfile(file, float, NF)
            temp_numP = np.fromfile(file, float, numParticles)
            temp = []
            temp = temp.append([temp_NF, temp_numP])
            pData = np.dstack(temp)
            del temp
        
    return



In [ ]:
filename='output_4_2__2_10mm_Rods.dat'
readBinDataSE(filename,1,50,20000)
print(pData.size)

## Plot Data
Plot the received data as a 2D and 3D animation. The particles should be color mapped according to their magnitude of velocity. 

In [ ]:
contents = io.loadmat('pData_mat.mat') #pData 1,50,20000
pData=contents['pData']
pData.shape

### Test 2D Scatter Plot, 1 Frame

In [ ]:
%matplotlib qt

frame = 48

zp=pData[3,:,frame] # note: x in pData is z for python
yp=pData[4,:,frame] # note: y in pData is y for python
xp=pData[5,:,frame] # note: z in pData is x for python

vp=pData[6,:,frame] #Importing zeroes :(
wp=pData[7,:,frame] 
up=pData[8,:,frame]

umag=np.sqrt(up**2+vp**2+wp**2)

fig = plt.figure()
ax = fig.add_subplot(111)

# Axes Properties 
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('2D Test: Time = %i' %frame)

ax.scatter(yp, zp, c=umag, cmap=cm.jet, s=0.1)
plt.show()

### Test 2D Scatter Plot, Animated

In [ ]:
%matplotlib qt

time = len(pData[0][0]) #Set the time frame

fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion() #Animate plot

for i in range(time):    
    zp=pData[3,:,i] # note: x in pData is z for python
    yp=pData[4,:,i] # note: y in pData is y for python
    xp=pData[5,:,i] # note: z in pData is x for python

    vp=pData[6,:,i]
    wp=pData[7,:,i]
    up=pData[8,:,i]
    umag=np.sqrt(up**2+vp**2+wp**2) # magnitude of velocity
    
    plt.scatter(yp,zp, s=.1, c=umag, cmap=cm.jet)
    plt.xlabel('Z')
    plt.ylabel('Y')
    plt.axis([0,1.0,0,1.0])
    plt.title('2D Animated Scatter: Time = %i' %i)
    plt.draw()
    plt.pause(0.2)
    plt.clf()
    
plt.waitforbuttonpress()

### Test 3D Scatter Plot, 1 Frame

In [ ]:
%matplotlib qt
# %matplotlib inline

frame = 48

zp=pData[3,:,frame] # note: x in pData is z for python
yp=pData[4,:,frame] # note: y in pData is y for python
xp=pData[5,:,frame] # note: z in pData is x for python

vp=pData[6,:,frame]
wp=pData[7,:,frame]
up=pData[8,:,frame]

umag=np.sqrt(up**2+vp**2+wp**2) # magnitude of velocity
    
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Axes Properties 
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.set_title('3D Test: Time = %i' %frame)
ax.view_init(20, 10)

ax.scatter(xp, yp, zp, c=umag, s=0.1)
plt.show()

### Test 3D Scatter Plot, Animated

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import pandas as pd


time = 3 # len(pData[0][0])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.ion()

def update_graph(i):
    zp=pData[3,:,i] # note: x in pData is z for python
    yp=pData[4,:,i] # note: y in pData is y for python
    xp=pData[5,:,i] # note: z in pData is x for python

    vp=pData[6,:,i]
    wp=pData[7,:,i]
    up=pData[8,:,i]
    umag=np.sqrt(up**2+vp**2+wp**2) # magnitude of velocity
    graph.offsets3d = (xp, yp, zp)
    
    ax.scatter(xp,yp,zp,c=umag,s=0.1)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('3D Test: Time = %i' %i)
    ax.view_init(20, 10)
    plt.draw()
    plt.pause(0.5)
    plt.clf()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.ion()

In [ ]:
#Creates only 1 succesful rest are 0 axes


%matplotlib inline

time = 5 #len(pData[0][0])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.ion()

for i in range(time):    
    zp=pData[3,:,i] # note: x in pData is z for python
    yp=pData[4,:,i] # note: y in pData is y for python
    xp=pData[5,:,i] # note: z in pData is x for python

    vp=pData[6,:,i]
    wp=pData[7,:,i]
    up=pData[8,:,i]
    umag=np.sqrt(up**2+vp**2+wp**2) # magnitude of velocity
    
    ax.scatter(xp,yp,zp,c=umag,s=0.1)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('3D Test: Time = %i' %i)
    ax.view_init(20, 10)
    plt.draw()
    plt.pause(0.5)
    plt.clf()

plt.waitforbuttonpress()


In [ ]:
# %matplotlib qt
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time

time = 49

contents = io.loadmat('pData_mat.mat') #pData 1,50,20000
pData=contents['pData']


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.ion()

for i in range(time):    
    zp=pData[3,:,i] # note: x in pData is z for python
    yp=pData[4,:,i] # note: y in pData is y for python
    xp=pData[5,:,i] # note: z in pData is x for python

    vp=pData[6,:,i]
    wp=pData[7,:,i]
    up=pData[8,:,i]
    umag=np.sqrt(up**2+vp**2+wp**2) # magnitude of velocity
    
    ax.scatter(xp,yp,zp,c=umag,s=0.1)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('3D Test: Time = %i' %time)
    ax.view_init(20, 10)
    plt.draw()
    plt.pause(0.5)
#     plt.clf()




In [ ]:
contents = io.loadmat('data.mat') #pData 1,50,20000
pData=contents['pData']

for k in range(49): 
# - get data for time k
# position in (x,y,z); x points to the right and z points upward
    yp=pData[3,:,k] # note: x in pData is z for python
    zp=pData[4,:,k] # note: y in pData is y for python
    xp=pData[5,:,k] # note: z in pData is x for python
# velocity vector (u,v,w) in (x,y,z)
    vp=pData[6]
    wp=pData[7]
    up=pData[8]
# now you can calculate something
    umag=np.sqrt(up**2+vp**2+wp**2) # magnitude of velocity
    
    dataSet = np.array([xp,yp,zp])
#     3D view
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('3D Test')
    
    line_ani = animation.FuncAnimation(fig, )


In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
import numpy as np

fig = plt.figure()
ax = p3.Axes3D(fig)

# Plot the first position for all particles
h = ax.plot(*data[0].T, marker='.', linestyle='None')[0]
# Equivalent to
# h = ax.plot(data[0, :, 0], data[0, :, 1], data[0, :, 2], 
#             marker='.', linestyle='None')[0]

# Setting the axes properties
ax.set_xlim3d([-100.0, 100.0])
ax.set_xlabel('X')

ax.set_ylim3d([-100.0, 100.0])
ax.set_ylabel('Y')

ax.set_zlim3d([-100.0, 100.0])
ax.set_zlabel('Z')
ax.set_title('3D Test')

def update_particles(num):
    # Plot the iterations up to num for all particles
    h.set_xdata(data[:num, :, 0].ravel())
    h.set_ydata(data[:num, :, 1].ravel())
    h.set_3d_properties(data[:num, :, 2].ravel())
    return h

prtcl_ani = animation.FuncAnimation(fig, update_particles, frames=301, 
                                    interval=10)
prtcl_ani